In [16]:
import pandas as pd
import numpy as np
df = pd.read_excel("2023 Pollution Inventory Dataset V2.xlsx",skiprows=9)
df.head()

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR
0,UP3133UK,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Chequer Tree Farm And Coursehorn Farm Poultry ...,TN17 3PN,578630.0,134230.0,"Kent, South London and East Sussex",Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
1,QP3133UM,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Winfarthing Farm Poultry Unit Short Green Winf...,IP22 2EE,610040.0,286300.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
2,UP3937FW,Animal vegetable and food treating etc vegetable,Cawingredients Limited,Caw House Conygarth Way Leeming Bar Business P...,DL7 9FD,428230.0,490060.0,Yorkshire,Controlled Waters,Alachlor,0.1,Below Reporting Threshold,Food & Drink,Food & Drink
3,BN2832IK,Animal vegetable and food treating etc vegetable,Britvic Soft Drinks Plc,Beckton Soft Drinks Factory 9 Roding Road Lond...,E6 6LF,NaN,NaN,Hertfordshire and North London,Wastewater,Alachlor,0.1,Below Reporting Threshold,Food & Drink,Food & Drink
4,AP3833US,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Martham East Farm Poultry Unit Rollesby Road M...,NR29 4SL,645750.0,317400.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming


In [17]:
#gases to be removed
remove_gases = [
    "Sulphur oxides (SO2 and SO3) as SO2", "Carbon monoxide", "Carbon dioxide", 
    "Carbon Dioxide From Qualifying Renewable Fuel Sources", "Methane", 
    "Nitrogen - as total N", "Nitrogen oxides (NO and NO2) as NO2", "Nitrous oxide", 
    "Sulphur hexafluoride", "Hydrogen cyanide", "Fluorine and inorganic fluorine compounds - as HF", 
    "Chlorofluorocarbons (CFCs)", "Hydrochlorofluorocarbons (HCFCs)", "Hydrofluorocarbons (HFCs)", 
    "Perfluorocarbons (PFCs)"]
#removing from dataset
df = df[~df["SUBSTANCE NAME"].isin(remove_gases)]

In [20]:
#drop rows where quantity released is 0 or "Below Reporting Threshold"
df = df[~df["QUANTITY RELEASED (kg)"].isin([0, "Below Reporting Threshold"])]

#dummy variables for certain columns
categorical_columns = ["EA AREA NAME", "ROUTE NAME", "REGULATED INDUSTRY SECTOR", "REGULATED INDUSTRY SUB SECTOR"]

for col in categorical_columns:
    unique_values = {value: idx for idx, value in enumerate(sorted(df[col].unique()))}
    df[col + " LABEL"] = df[col].map(unique_values)

#reset index after dropping rows
df.reset_index(drop=True, inplace=True)
df.head()

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR,EA AREA NAME LABEL,ROUTE NAME LABEL,REGULATED INDUSTRY SECTOR LABEL,REGULATED INDUSTRY SUB SECTOR LABEL
0,HP3634CZ,"INTENSIVE FARMING; > 40,000 POULTRY",Watkins; Watkins,Stonehouse Farm Kempley Dymock Gloucestershire,GL18 2AT,367380.0,231840.0,West Midlands,Air,Ammonia,1000.0000,816,Agriculture,Intensive Farming,12,0,0,8
1,BT5873IU,"INTENSIVE FARMING; > 40,000 POULTRY",LJ FAIRBURN AND SON LTD,Ivy House Farm Poultry Unit Ivy House Farm Far...,LN13 9PL,546500.0,374790.0,Lincolnshire and Northamptonshire,Air,Ammonia,1000.0000,898,Agriculture,Intensive Farming,7,0,0,8
2,XP3539RM,"INTENSIVE FARMING; > 40,000 POULTRY",Ridgeway Foods Limited,Seisdon Poultry Unit Fox Road Shipley West Mid...,WV6 7EL,382570.0,295320.0,West Midlands,Air,Ammonia,1000.0000,1067,Agriculture,Intensive Farming,12,0,0,8
3,XP3836GW,"INTENSIVE FARMING; > 40,000 POULTRY",Anderson; Anderson,Brook Farm Poultry Unit 5 Brook Lane Brookvill...,IP26 4RQ,573200.0,296200.0,East Anglia,Air,Ammonia,1000.0000,1080,Agriculture,Intensive Farming,2,0,0,8
4,FP3935KL,THE INCINERATION OF HAZARDOUS WASTE IN AN INCI...,TRADEBE FAWLEY LIMITED,Fawley High Temperature Incinerator Charlston ...,SO45 3NX,443260.0,105710.0,Solent and South Downs,Controlled Waters,Aldrin,0.0005,0.0028,EfW,EfW,9,1,5,3


In [19]:
print(len(df))
print(df.columns)

5987
Index(['AUTHORISATION ID / PERMIT ID', 'ACTIVITY DESCRIPTION', 'OPERATOR NAME',
       'SITE ADDRESS', 'SITE POSTCODE', 'EASTING', 'NORTHING', 'EA AREA NAME',
       'ROUTE NAME', 'SUBSTANCE NAME', 'REPORTING THRESHOLD (kg)',
       'QUANTITY RELEASED (kg)', 'REGULATED INDUSTRY SECTOR',
       'REGULATED INDUSTRY SUB SECTOR', 'EA AREA NAME LABEL',
       'ROUTE NAME LABEL', 'REGULATED INDUSTRY SECTOR LABEL',
       'REGULATED INDUSTRY SUB SECTOR LABEL'],
      dtype='object')
